In [1]:
# Dependencies to Visualize the model
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Filepaths, numpy, and Tensorflow
import os
import numpy as np
import keras

# Preprocessing imports
import glob
import os.path as path
import imageio
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# Model Imports
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import accuracy_score, f1_score

from datetime import datetime

Using TensorFlow backend.


In [2]:
# Load the images
file_paths = []
labels = []
targets = []

emotions = ['angry','disgust','fear','happy','neutral','sad','surprise']
x=0
e=1
for emotion in emotions:
    for filename in glob.glob(f"samples/{emotion}/*.jpg"):
        path = filename.split('\\')[0]+'/'+filename.split("\\")[1]
        file_paths.append(path)
        labels.append(e)
        targets.append(emotion)
        x+=1
    for filename in glob.glob(f"samples/{emotion}/*.png"):
        path = filename.split('\\')[0]+'/'+filename.split("\\")[1]
        file_paths.append(path)
        labels.append(e)
        targets.append(emotion)
        x+=1
    e+=1
# Load the images
images = []
image_size = (350,350)

for path in file_paths:
    img = load_img(path,target_size = image_size)
    img = img_to_array(img)
    images.append(img)

images = np.asarray(images)       
len(images)        

242

In [3]:
# Get image size
image_size = np.asarray([images.shape[1], images.shape[2], images.shape[3]])
print(image_size)

[350 350   3]


In [4]:
# Scale
images = images / 255

#Split into training and testing data
X = images
y = labels
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.shape

(181, 350, 350, 3)

In [5]:
from keras.utils import to_categorical

# Prepare y data
y_binary_train = to_categorical(y_train)
y_binary_test = to_categorical(y_test)

In [24]:
# Define hyperparamters
MIN_NEURONS = 20
MAX_NEURONS = 120
KERNEL = (3, 3)
n_layers = 4

# Training hyperparamters
EPOCHS = 150
BATCH_SIZE = 20
PATIENCE = 10

# Determine the # of neurons in each convolutional layer
steps = np.floor(MAX_NEURONS / (n_layers + 1))
nuerons = np.arange(MIN_NEURONS, MAX_NEURONS, steps)
nuerons = nuerons.astype(np.int32)

# Define a model
model = Sequential()

# Add convolutional layers
for i in range(0, n_layers):
    if i == 0:
        shape = (image_size[0], image_size[1], image_size[2])
        model.add(Conv2D(nuerons[i], KERNEL, input_shape=shape))
    else:
        model.add(Conv2D(nuerons[i], KERNEL))

    model.add(Activation('relu'))

# Add max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(MAX_NEURONS))
model.add(Activation('relu'))

# Add output layer
model.add(Dense(1))
model.add(Activation('softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [25]:
# Print a summary of the model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 348, 348, 20)      560       
_________________________________________________________________
activation_19 (Activation)   (None, 348, 348, 20)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 346, 346, 44)      7964      
_________________________________________________________________
activation_20 (Activation)   (None, 346, 346, 44)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 344, 344, 68)      26996     
_________________________________________________________________
activation_21 (Activation)   (None, 344, 344, 68)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 342, 342, 92)      56396     
__________

In [26]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=PATIENCE, verbose=2, mode='auto')

In [27]:
# TensorBoard callback
LOG_DIRECTORY_ROOT = ''
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
log_dir = "{}/run-{}/".format(LOG_DIRECTORY_ROOT, now)
tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True)

In [28]:
# Place the callbacks in a list
callbacks = [early_stopping, tensorboard]

In [29]:
# Train the model
model.fit(np.array(X_train), np.array(y_train), epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks, verbose=2)

ResourceExhaustedError: OOM when allocating tensor of shape [2690172,120] and type float
	 [[node training_2/Adam/zeros_8 (defined at C:\Users\bethf\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:702) ]]

Original stack trace for 'training_2/Adam/zeros_8':
  File "C:\Users\bethf\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\bethf\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\bethf\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\bethf\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\bethf\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\bethf\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C

In [31]:
# Make a prediction on the test set
test_predictions = model.predict(X_test)
test_predictions = np.round(test_predictions)

FailedPreconditionError: 2 root error(s) found.
  (0) Failed precondition: Attempting to use uninitialized value dense_8/bias
	 [[{{node dense_8/bias/read}}]]
	 [[activation_24/Softmax/_359]]
  (1) Failed precondition: Attempting to use uninitialized value dense_8/bias
	 [[{{node dense_8/bias/read}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
# Report the accuracy
accuracy = accuracy_score(y_test, test_predictions)
print("Accuracy: " + str(accuracy))